# [WIP] Prediction using rainfall data

The best model without rainfall data is Mean Daily Pattern Model.
In this notebook we try to improve this model by using precipitation information.

In [11]:
import datetime
import calendar
import json
import numpy as np
import pandas as pd
from sklearn import tree
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.figsize'] = 12, 4

# Load project

In [2]:
project_folder = '../../datasets/thorium-medium/'
with open(project_folder + 'project.json', 'r') as file:
    project = json.load(file)
print(json.dumps(project, indent=4))
flow = pd.read_csv(project_folder + 'flow1.csv', parse_dates=['time'])
flow = flow.set_index('time')['flow'].fillna(0)
flow = flow.resample('5T').pad()
rainfall = pd.read_csv(project_folder + 'rainfall1.csv', parse_dates=['time'])
rainfall = rainfall.set_index('time')['rainfall'].fillna(0)
rainfall = rainfall.resample('5T').pad()
flow_rain = pd.concat([flow, rainfall], axis=1).dropna()
print(flow_rain.head())
print(flow_rain.tail())

{
    "start-date": "2015-06-02",
    "split-date": "2017-01-01",
    "end-date": "2017-10-01",
    "name": "thorium-medium",
    "flows": [
        "flow1"
    ],
    "rainfalls": [
        "rainfall1"
    ]
}
                           flow  rainfall
time                                     
2015-06-01 14:15:00  115.559998       0.0
2015-06-01 14:20:00  115.199997       0.0
2015-06-01 14:25:00  112.209999       0.0
2015-06-01 14:30:00  112.860001       0.0
2015-06-01 14:35:00  113.349998       0.0
                           flow  rainfall
time                                     
2017-11-10 14:20:00  107.830002       0.0
2017-11-10 14:25:00  107.459999       0.0
2017-11-10 14:30:00  106.919998       0.0
2017-11-10 14:35:00  105.559998       0.0
2017-11-10 14:40:00  104.940002       0.0


## Helper functions

Helper functions for building training and test sets and calculating score

In [18]:
class PredictionModel:
    
    def fit(self, flow, rain):
        """ Use daily pattern """
        df = flow.to_frame().reset_index()
        self.daily_pattern = df.groupby(by=[df.time.map(lambda x : (x.hour, x.minute))]).flow.mean().values
        
    def predict(self, day, rain):
        return self.daily_pattern

    
def loss(y_hat, y):
    """
    https://en.wikipedia.org/wiki/Mean_absolute_percentage_error
    """
    return 100.0 * np.sum(np.abs((y-y_hat) / y)) / y.shape[0]


def split_data(flow, split_day):
    """Get all data up to given day"""
    end_day = split_day - pd.Timedelta('1 min')
    return flow[:end_day]


def evaluate_day(model, flow, rain, split_day):
    """Evaluate data for single day"""
    xs = split_data(flow, split_day)
    next_day = split_day + pd.Timedelta(1, 'D')
    y = flow[next_day: next_day+pd.Timedelta('1439 min')]
    model.fit(xs, rain)
    y_hat = model.predict(next_day, rain)
    return loss(y_hat, y)


def evaluate_model(model, flow, rain, start_day):
    """
    Evaluate model on all days starting from split_day.
    Returns 90th percentile error as model score
    """
    last_day = flow.index[-1] - pd.Timedelta(1, 'D')
    split_day = start_day
    costs = []
    while split_day < last_day:
        cost = evaluate_day(model, flow, rain, split_day)
        costs.append(cost)
        split_day += pd.Timedelta(1, 'D')
    return np.mean(costs), costs

evaluate_day(PredictionModel(), flow_rain.flow, flow_rain.rainfall, pd.Timestamp('2016-11-10'))

8.7293481901433569

# Extract features

Extract the following features:

  * Minut of the day
  * Total precipitation in the last N hours

In [20]:
def encode_time(xs):
    """
    Encode time as Int value
    Params:
      times - pd.Series with Timestamp
    Return:
      pd.Series with encoded time
    """
    return xs.map(lambda x: x.hour*60+x.minute)


def encode_features(flow, rain, last_rain_window = 2):
    """
    Create feature vector based on 
    Return feature and expected values
    """
    df = pd.concat([flow, rain], axis=1).reset_index()
    df['last_rain'] = df.rainfall.rolling(last_rain_window*12).sum()
    df['minutes'] = encode_time(df.time)
    df = df.dropna()
    return (df[['minutes', 'last_rain']], df.flow)


def prepare_prediction_features(day, rain, last_rain_window = 2):
    last_rain = rain.rolling(last_rain_window*12).sum()
    ts = pd.date_range(day, periods=288, freq='5T')
    minutes = pd.Series(data=encode_time(ts), index=ts)
    df = pd.concat([minutes, last_rain], axis=1).dropna()
    return df

# Models

## Linear regression

As a baseline lets try Linear Model

In [23]:
from sklearn.linear_model import LinearRegression

class LinearModel(PredictionModel):
    
    def fit(self, flow, rain):
        X, y = encode_features(flow, rain)
        self.clf = LinearRegression()
        self.clf.fit(X, y)
        
    def predict(self, day, rain):
        X = prepare_prediction_features(day, rain)
        return self.clf.predict(X)    

score, costs = evaluate_model(LinearModel(), flow_rain.flow, flow_rain.rainfall, pd.Timestamp('2017-01-01'))
print('LinearModel score: {:.2f}%'.format(score))

LinearModel score: 13.73


## Decision Tree Regressor

First not linear model. Should improve on linear model

In [24]:
from sklearn import tree

class DTModel(PredictionModel):
    
    def __init__(self, rain_window_size=2):
        self.rain_window_size = rain_window_size
    
    def fit(self, flow, rain):
        X, y = encode_features(flow, rain, self.rain_window_size)
        self.clf = tree.DecisionTreeRegressor()
        self.clf.fit(X, y)
        
    def predict(self, day, rain):
        X = prepare_prediction_features(day, rain, self.rain_window_size)
        return self.clf.predict(X)
    

score, costs = evaluate_model(DTModel(2), flow_rain.flow, flow_rain.rainfall, pd.Timestamp('2017-01-01'))
print('DTModel 2h score: {:.2f}%'.format(score))

DTModel 2h score: 9.24%


## XGBoost

In [ ]:
import xgboost as xg

class XGBoostModel(PredictionModel):
    
    def __init__(self, rain_window_size=2):
        self.rain_window_size = rain_window_size
    
    def fit(self, flow, rain):
        X, y = encode_features(flow, rain, self.rain_window_size)
        self.clf = xg.XGBRegressor()
        self.clf.fit(X.values, y.values)
        
    def predict(self, day, rain):
        X = prepare_prediction_features(day, rain, self.rain_window_size)
        return self.clf.predict(X.values)
    
    def encode_time(self, time):
        return time.hour*60+time.minute
    

score, costs = evaluate_model(XGBoostModel(2), flow_rain.flow, flow_rain.rainfall, pd.Timestamp('2017-01-01'))
print('XGBoostModel 2h score: {:.2f}'.format(score))